# 3.6 Sentiment Analysis

In [1]:
#import packages

import pandas as pd
from textblob_de import TextBlobDE as TextBlob
import numpy as np
from tqdm.notebook import tqdm

import re
import pickle
pd.options.mode.chained_assignment = None  # default='warn' based on false positives
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab
from spacy_sentiws import spaCySentiWS
from spacy_sentiws import spaCySentiWS


tqdm.pandas()

#load in the preprocessed data

pre_data_twitter= pickle.load(open('../data/processed/tweets_processed.p','rb'))[0:100]
pre_data_speeches= pickle.load(open('../data/processed/speeches_processed.p','rb'))[0:100]
pre_data_twitter.head()

,full_name,date,party,text_preprocessed,text_preprocessed_sentence,like_count
0,Ralph Brinkhaus,2021-06-15,CDU,"[fußballfans, freuen, spiel, nationalmannschaf...",fußballfans freuen spiel nationalmannschaft dr...,32.0
1,Ralph Brinkhaus,2021-06-11,CDU,"[außenpolitik, wirtschaftlich, souveränität, d...",außenpolitik wirtschaftlich souveränität digit...,5.0
2,Ralph Brinkhaus,2021-06-11,CDU,"[nachhaltig, klimawandel, kämpfen, brauchen, a...",nachhaltig klimawandel kämpfen brauchen akzept...,4.0
3,Ralph Brinkhaus,2021-06-11,CDU,"[brauchen, pandemie, bezahlen, arbeitsplätze, ...",brauchen pandemie bezahlen arbeitsplätze digit...,2.0
4,Ralph Brinkhaus,2021-06-11,CDU,"[wahldebatte, thema, zukunft, passieren, coron...",wahldebatte thema zukunft passieren corona sta...,24.0


## 3.6.1 Sentiment Analysis with TextBlob

### 3.6.1.1 Sentiment Analysis for Twitter Data

In [2]:
#loop through all the politicians we want to analyze
data=[]
for name in tqdm(['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Graf Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas' , 'Martin Schulz' , 'Karamba Diaby' , 'Karl Lauterbach',
            'Steffi Lemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin von Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , 'Stephan Brandner' , 'Tino Chrupalla' , 'Götz Frömming' , 'Leif-Erik Holm']):
    #get tweets from the specific politician 
    tweets_analyzing =pre_data_twitter.loc[pre_data_twitter['full_name']==name]
    #create sentiment scores
    blobs=tweets_analyzing['text_preprocessed_sentence'].apply(TextBlob)
    sentiment=[]
    for blob in blobs:
        sentiment.append(blob.sentiment)
    #get the polarity scores
    polarity=[]
    for egg in sentiment:
        polarity.append(egg.polarity)
    #get the mean of the scores 
    p_mean = np.mean(polarity)
    #get the number of positive, neutral and negative tweets
    positive_p=0
    neutral_p=0
    negative_p=0
    for item_p in polarity:
        if item_p>0:
            positive_p += 1
        elif item_p<0:
            negative_p += 1
        else:
            neutral_p += 1
    #set up list to secure the values generated
    data.append([name,p_mean,positive_p,neutral_p,negative_p]) 

  0%|          | 0/42 [00:00<?, ?it/s]

/Users/stjepankusenic/.conda/envs/amca/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/stjepankusenic/.conda/envs/amca/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
#set up dataframe with all values and save it into a csv file
dataf = pd.DataFrame(data, columns=['Name','Polarity_mean','Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets'])
dataf.to_csv('../data/processed/sentiment_scores_twitter_01.csv')
dataf.head()

,Name,Polarity_mean,Num_pos_tweets,Num_neutral_tweets,Num_neg_tweets
0,Ralph Brinkhaus,0.265083,42,48,10
1,Hermann Gröhe,NaN,0,0,0
2,Nadine Schön,NaN,0,0,0
3,Norbert Röttgen,NaN,0,0,0
4,Peter Altmaier,NaN,0,0,0


In [4]:
#create a polarity column for our dataset
blobs=pre_data_twitter['text_preprocessed_sentence'].progress_apply(TextBlob)
sentiment=[]
for blob in blobs:
    sentiment.append(blob.sentiment)
#get the scores
polarity=[]
for egg in sentiment:
    polarity.append(egg.polarity)
pre_data_twitter['polarity_textblob'] = polarity

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
display(pre_data_twitter)
    

,full_name,date,party,text_preprocessed,text_preprocessed_sentence,like_count,polarity_textblob
0,Ralph Brinkhaus,2021-06-15,CDU,"[fußballfans, freuen, spiel, nationalmannschaf...",fußballfans freuen spiel nationalmannschaft dr...,32.0,0.0
1,Ralph Brinkhaus,2021-06-11,CDU,"[außenpolitik, wirtschaftlich, souveränität, d...",außenpolitik wirtschaftlich souveränität digit...,5.0,0.0
2,Ralph Brinkhaus,2021-06-11,CDU,"[nachhaltig, klimawandel, kämpfen, brauchen, a...",nachhaltig klimawandel kämpfen brauchen akzept...,4.0,1.0
3,Ralph Brinkhaus,2021-06-11,CDU,"[brauchen, pandemie, bezahlen, arbeitsplätze, ...",brauchen pandemie bezahlen arbeitsplätze digit...,2.0,-1.0
4,Ralph Brinkhaus,2021-06-11,CDU,"[wahldebatte, thema, zukunft, passieren, coron...",wahldebatte thema zukunft passieren corona sta...,24.0,0.0
...,...,...,...,...,...,...,...
95,Ralph Brinkhaus,2020-10-31,CDU,"[vorfreude, jörg, jakob, gast, live, dopa]",vorfreude jörg jakob gast live dopa,33.0,0.0
96,Ralph Brinkhaus,2020-10-30,CDU,"[einschalten, gemeinsam, gründer, markus, luka...",einschalten gemeinsam gründer markus lukasson ...,24.0,0.0
97,Ralph Brinkhaus,2020-10-29,CDU,"[maßnahme, eindämmung, pandemie, sprechen, maß...",maßnahme eindämmung pandemie sprechen maßnahme...,33.0,0.0
98,Ralph Brinkhaus,2020-10-29,CDU,"[virus, warten, unterhalten, leistungsfähig, s...",virus warten unterhalten leistungsfähig sagen ...,132.0,0.9


### 3.6.1.2 Sentiment Analysis Bundestag Speeches

In [6]:
#loop through all the politicians we want to analyze
data=[]
for name in tqdm(['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Graf Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas' , 'Martin Schulz' , 'Karamba Diaby' , 'Karl Lauterbach',
            'Steffi Lemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin von Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , 'Stephan Brandner' , 'Tino Chrupalla' , 'Götz Frömming' , 'Leif-Erik Holm']):
    #get speeches from the specific politician
    speeches_analyzing =pre_data_speeches.loc[pre_data_speeches['full_name']==name]
    #create sentiment scores
    blobs=speeches_analyzing['text_preprocessed_sentence'].apply(TextBlob)
    sentiment=[]
    for blob in blobs:
        sentiment.append(blob.sentiment)
    #get the polarity scores
    polarity=[]
    for egg in sentiment:
        polarity.append(egg.polarity)
    #get the mean and of the polarity values 
    p_mean = np.mean(polarity)
    #get the number of positive, neutral and negative tweets
    positive_p=0
    neutral_p=0
    negative_p=0
    for item_p in polarity:
        if item_p>0:
            positive_p += 1
        elif item_p<0:
            negative_p += 1
        else:
            neutral_p += 1
    #set up list to secure the values generated
    data.append([name,p_mean,positive_p,neutral_p,negative_p]) 

  0%|          | 0/42 [00:00<?, ?it/s]

/Users/stjepankusenic/.conda/envs/amca/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/stjepankusenic/.conda/envs/amca/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
#set up dataframe with all values
dataf = pd.DataFrame(data, columns=['Name','Polarity_mean','Num_pos_speeches','Num_neutral_speeches','Num_neg_speeches'])
dataf.to_csv('../data/processed/sentiment_scores_speeches_01.csv')
dataf.head()

,Name,Polarity_mean,Num_pos_speeches,Num_neutral_speeches,Num_neg_speeches
0,Ralph Brinkhaus,0.435714,1,0,0
1,Hermann Gröhe,NaN,0,0,0
2,Nadine Schön,0.207771,3,1,1
3,Norbert Röttgen,-0.421739,0,1,2
4,Peter Altmaier,0.532353,1,0,0


In [8]:
blobs=pre_data_speeches['text_preprocessed_sentence'].apply(TextBlob)
sentiment=[]
for blob in blobs:
    sentiment.append(blob.sentiment)
#get the scores
polarity=[]
for egg in sentiment:
    polarity.append(egg.polarity)
pre_data_speeches['polarity_textblob'] = polarity

In [9]:
display(pre_data_speeches)

,full_name,date,party,text_preprocessed,text_preprocessed_sentence,polarity_textblob
1114,Jan Korte,2017-10-24,Linke,"[herr, präsident, lieben, kollegin, kollege, g...",herr präsident lieben kollegin kollege geehrt ...,0.512121
1115,Marco Buschmann,2017-10-24,FDP,"[herr, präsident, lieb, kollegin, kollege, kon...",herr präsident lieb kollegin kollege konstitui...,0.061111
1116,Britta Haßelmann,2017-10-24,Grüne,"[geehrt, herr, präsident, dame, herr, kern, de...",geehrt herr präsident dame herr kern debatte t...,0.231579
1117,Marco Buschmann,2017-11-21,FDP,"[herr, präsident, geehrt, kollegin, kollege, f...",herr präsident geehrt kollegin kollege fraktio...,0.750000
1118,Jan Korte,2017-11-21,Linke,"[geehrt, herr, präsident, dame, herr, ernst, z...",geehrt herr präsident dame herr ernst zeit hum...,0.293333
...,...,...,...,...,...,...
1210,Tino Chrupalla,2018-01-31,AFD,"[frau, ministerin, laut, jahresgutachten, jung...",frau ministerin laut jahresgutachten junge men...,0.850000
1211,Leif-Erik Holm,2018-02-02,AFD,"[lieben, bürger, herr, präsident, dame, herr, ...",lieben bürger herr präsident dame herr freuen ...,0.097059
1212,Leif-Erik Holm,2018-01-31,AFD,"[frau, präsidentin, frage, frau, wirtschaftsmi...",frau präsidentin frage frau wirtschaftsministe...,0.000000
1213,Bettina Stark-Watzinger,2018-01-31,FDP,"[sondierungspapier, erinnern, satz, mark, twai...",sondierungspapier erinnern satz mark twain ric...,-0.700000


## 3.6.2 Sentiment Analysis with SentiWS

In [14]:
#insert pipeline to add sentiws preprocessing

In [15]:
pre_data_twitter= pd.read_csv("../data/raw/tweets_explored.csv")[0:100]
pre_data_speeches= pd.read_csv("../data/raw/speeches_explored.csv")[0:100]

In [16]:
@Language.component("Remove non alphabetic words")
def remove_non_alpha(doc):
    return [token for token in doc if token.is_alpha]

In [17]:
@Language.factory("Detect languages")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)

In [18]:
@Language.factory("Sentiment Appplication")
def create_sentiment_dictionary(nlp, name):
    return spaCySentiWS(sentiws_path = "../data/raw/Sentiment/")

In [19]:
@Language.component("Keep only German documents")
def remove_non_german(doc):
    res = [sent for sent in doc.sents if sent._.language["language"] == "de"]
    if res:
        return [token for sent in res for token in sent]
    else:
        return Doc(Vocab([]), words=[], spaces=[])

In [20]:
@Language.component("Remove stopwords")
def remove_stopwords(doc): 
    return [token for token in doc if not token.is_stop]

In [21]:
@Language.component("Lemmatize text")
def lemmatize_text(doc):
    return [token.lemma_ for token in doc]

In [22]:
@Language.component("Lowercase Text")
def lowercase(doc):
    return [token.lower() for token in doc]

In [23]:
emoji_codes = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", re.UNICODE)

@Language.component("Remove emojis")
def remove_emojis(doc):
    doc = [token.text for token in doc if not re.match(emoji, token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

In [24]:
@Language.component("Remove URLs")
def remove_urls(doc):
    doc = [token.text for token in doc if not token.like_url]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

In [25]:
@Language.component("Remove mentions")
def remove_mentions(doc):
    doc = [token.text for token in doc if not re.match("@.*", token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

In [26]:
@Language.component("Remove stopwords and punctuation")
def remove_stopwords(doc):
    doc = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return doc

In [27]:
#pipeline_exclude = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'morphologizer']

In [28]:
# Create spacy pipeline
nlp_tweets_sentiws = spacy.load('de_core_news_sm')
nlp_tweets_sentiws.Defaults.stop_words |= {"amp", "rt"}

# The add_pipe function appends our functions to the default pipeline.
nlp_tweets_sentiws.add_pipe("sentencizer", last=True)
nlp_tweets_sentiws.add_pipe("Detect languages", name='Detect languages', last=True)
nlp_tweets_sentiws.add_pipe("Keep only German documents", name='Keep only German documents', last=True)
nlp_tweets_sentiws.add_pipe("Remove non alphabetic words", name="Remove non alphabetic words", last=True)
nlp_tweets_sentiws.add_pipe("Remove stopwords", name="Remove stopwords", last=True)
# nlp_tweets.add_pipe("Lemmatize text", name="Lemmatize text", last=True)
# nlp_tweets.add_pipe("Lowercase Text", name="Lowercase Text", last=True)
nlp_tweets_sentiws.add_pipe("Sentiment Appplication", name="Sentiment Appplication", last=True)

### 3.6.2.1 Sentiment Analysis for Twitter Data

In [29]:
#Apply the sentiment anaylsis to the Twitter accounts of the politicians
data=[]
for name in tqdm(['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Graf Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas' , 'Martin Schulz' , 'Karamba Diaby' , 'Karl Lauterbach',
            'Steffi Lemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin von Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , 'Stephan Brandner' , 'Tino Chrupalla' , 'Götz Frömming' , 'Leif-Erik Holm']):
    #get tweets from the specific politician
    tweets_analyzing = pre_data_twitter.loc[pre_data_twitter['full_name']==name]
    tweets_analyzing1 = tweets_analyzing.text.progress_apply(nlp_tweets_sentiws)
    #get the sentiment of the tweets
    politician_sum=[]
    for sentence in tweets_analyzing1:
        sentence_sum=[]
        for token in sentence:
            if token._.sentiws == None:
                a=0
            elif token._.sentiws == 'nan':
                a=0
            else:
                sentence_sum.append(token._.sentiws)
        sentence_score=np.nanmean(sentence_sum)
        politician_sum.append(sentence_score)
    politician_score=np.nanmean(politician_sum)
    #get the number of positive, neutral and negative tweets
    positive_p=0
    neutral_p=0
    negative_p=0
    for item_p in politician_sum:
        if item_p>0:
            positive_p += 1
        elif item_p<0:
            negative_p += 1
        elif item_p == 'nan':
            neutral_p += 1
        else:
            neutral_p += 1
    #set up list to secure the values generated
    data.append([name,politician_score,positive_p,neutral_p,negative_p])

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_19030/393194382.py:23: RuntimeWarning: Mean of empty slice
  sentence_score=np.nanmean(sentence_sum)


0it [00:00, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_19030/393194382.py:25: RuntimeWarning: Mean of empty slice
  politician_score=np.nanmean(politician_sum)


...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [30]:
#set up dataframe with all values
dataf = pd.DataFrame(data, columns=['Name','Polarity_mean','Num_pos_tweets','Num_neutral_tweets','Num_neg_tweets'])
dataf.to_csv('../data/processed/sentiment_scores_tweets_sentiws_01.csv')
dataf.head()

,Name,Polarity_mean,Num_pos_tweets,Num_neutral_tweets,Num_neg_tweets
0,Ralph Brinkhaus,0.02611,48,27,25
1,Hermann Gröhe,NaN,0,0,0
2,Nadine Schön,NaN,0,0,0
3,Norbert Röttgen,NaN,0,0,0
4,Peter Altmaier,NaN,0,0,0


### 3.6.2.2 Sentiment Analysis for Bundestag Speeches

In [31]:
#Apply the sentiment analysis to the speeches accounts of the politicians
data=[]
for name in tqdm(['Ralph Brinkhaus','Hermann Gröhe', 'Nadine Schön' ,'Norbert Röttgen' , 'Peter Altmaier' , 'Jens Spahn' , 'Matthias Hauer',
            'Christian Lindner' , 'Marco Buschmann' , 'Bettina Stark-Watzinger', 'Alexander Graf Lambsdorff' , 'Johannes Vogel' , 'Konstantin Kuhle' , 'Marie-Agnes Strack-Zimmermann',
            'Lars Klingbeil' , 'Saskia Esken' , 'Hubertus Heil' , 'Heiko Maas' , 'Martin Schulz' , 'Karamba Diaby' , 'Karl Lauterbach',
            'Steffi Lemke' , 'Cem Özdemir' , 'Katrin Göring-Eckardt' , 'Konstantin von Notz' , 'Britta Haßelmann' , 'Sven Lehmann' , 'Annalena Baerbock',
            'Sahra Wagenknecht' , 'Bernd Riexinger' , 'Niema Movassat' , 'Jan Korte' , 'Dietmar Bartsch' , 'Gregor Gysi' , 'Sevim Dağdelen',
            'Alice Weidel' , 'Beatrix von Storch' , 'Joana Cotar' , 'Stephan Brandner' , 'Tino Chrupalla' , 'Götz Frömming' , 'Leif-Erik Holm']):
    #get speeches from the specific politician
    speeches_analyzing = pre_data_speeches.loc[pre_data_speeches['full_name']==name]
    speeches_analyzing1 = speeches_analyzing.text.progress_apply(nlp_tweets_sentiws)
    #get the sentiment of the tweets
    politician_sum=[]
    for sentence in speeches_analyzing1:
        sentence_sum=[]
        for token in sentence:
            if token._.sentiws == None:
                a=0
            elif token._.sentiws == 'nan':
                a=0
            else:
                sentence_sum.append(token._.sentiws)
        sentence_score=np.nanmean(sentence_sum)
        politician_sum.append(sentence_score)
    politician_score=np.nanmean(politician_sum)
    #get the number of positive, neutral and negative tweets
    positive_p=0
    neutral_p=0
    negative_p=0
    for item_p in politician_sum:
        if item_p>0:
            positive_p += 1
        elif item_p<0:
            negative_p += 1
        elif item_p == 'nan':
            neutral_p += 1
        else:
            neutral_p += 1
    #set up list to secure the values generated
    data.append([name,politician_score,positive_p,neutral_p,negative_p])

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [32]:
#set up dataframe with all values
dataf = pd.DataFrame(data, columns=['Name','Polarity_mean','Num_pos_speeches','Num_neutral_speeches','Num_neg_speeches'])
dataf.to_csv('../data/processed/sentiment_scores_speeches_sentiws_01.csv')
dataf.head()

,Name,Polarity_mean,Num_pos_speeches,Num_neutral_speeches,Num_neg_speeches
0,Ralph Brinkhaus,-0.024537,0,0,2
1,Hermann Gröhe,0.082749,3,0,1
2,Nadine Schön,-0.042718,1,0,1
3,Norbert Röttgen,-0.099983,0,1,2
4,Peter Altmaier,0.140338,3,2,0
